In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings(action= 'ignore')

StatementMeta(, cc516c66-db08-4daf-8e0e-e580de23a65f, 3, Finished, Available)

en un principio se intenta automatizar la conversion de archivos tipo .json a archivos .csv que son mas
optimos para trabajar en el entorno de power query

para esto tomaremos la totalidad de los ficheros dentro de la carpeta CENSOS de nuestro lakehouse y a traves de una iteracion los guardaremos en la carpeta data final, para una vez alli realizar a traves de la canalizacion de datos (pipeline) la copia de los archivos que nos interesen hacia nuestro lakehouse destino

a su vez, una vez realizado este paso debemos automatizar la posibilidad a futuro de nuevas conversiones (ya que se estima que se realizara una automatizacion de la extraccion de datos de nuestra fuente externa cuando esten disponibles segun el criterio que tomemos (en este caso el año), en caso contrario, no se extraeran nuevos datos)

en ese caso, el paso de copia de nuevos ficheros hacia el lakehouse destino no se realizara ya que no se dispone de nueva data

In [2]:
# tomemos un fichero como muestra para ver como es la disposicion de los datos de nuestros archivos .json

df = pd.read_json('abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/37a0382d-f783-4728-a697-48b1dc002937/Files/CENSO/Current_Population_Survey2022.json')

StatementMeta(, 2dfd7a0e-b9a8-441b-90b6-0279ec361be2, 4, Finished, Available)

In [16]:
df.head()

StatementMeta(, 11524b49-fb37-4da6-b5d0-ef3ddccd48c0, 18, Finished, Available)

,HRNUMHOU,PTERNH1O,PEMNTVTY,PEFNTVTY,GTCBSASZ,GESTFIPS,GTCSA,PRCHLD,PRTAGE,PESEX,GTCBSA,HEFAMINC,HRHTYPE,state
0,1,-0.01,57,57,3,1,0,0,49,2,26620,12,7,1
1,1,-0.01,57,57,4,5,340,0,67,1,30780,8,6,5
2,2,-0.01,57,57,4,5,340,0,72,2,30780,7,1,5
3,2,-0.01,57,57,4,5,340,0,85,1,30780,7,1,5
4,4,-0.01,57,57,5,40,0,8,34,2,36420,1,4,40


In [17]:
df.info()

StatementMeta(, 11524b49-fb37-4da6-b5d0-ef3ddccd48c0, 19, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Index: 105630 entries, 0 to 105629
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   HRNUMHOU  105630 non-null  int64  
 1   PTERNH1O  105630 non-null  float64
 2   PEMNTVTY  105630 non-null  int64  
 3   PEFNTVTY  105630 non-null  int64  
 4   GTCBSASZ  105630 non-null  int64  
 5   GESTFIPS  105630 non-null  int64  
 6   GTCSA     105630 non-null  int64  
 7   PRCHLD    105630 non-null  int64  
 8   PRTAGE    105630 non-null  int64  
 9   PESEX     105630 non-null  int64  
 10  GTCBSA    105630 non-null  int64  
 11  HEFAMINC  105630 non-null  int64  
 12  HRHTYPE   105630 non-null  int64  
 13  state     105630 non-null  int64  
dtypes: float64(1), int64(13)
memory usage: 12.1 MB


In [2]:
#veamos los archivos que existen dentro de la carpeta censo dentro de data final
ruta_carpeta_destino = '/lakehouse/default/Files/DATA FINAL/CENSO'
archivos_destino = os.listdir(ruta_carpeta_destino)
print(archivos_destino)

StatementMeta(, , , Waiting, )

[]


In [3]:
#veamos los archivos que existen dentro de la carpeta censo
ruta_carpeta_origen = '/lakehouse/default/Files/CENSO'
archivos_origen = os.listdir(ruta_carpeta_origen)
print(archivos_origen)

StatementMeta(, , , Waiting, )

['Current_Population_Survey2011.json', 'Current_Population_Survey2012.json', 'Current_Population_Survey2013.json', 'Current_Population_Survey2014.json', 'Current_Population_Survey2015.json', 'Current_Population_Survey2016.json', 'Current_Population_Survey2017.json', 'Current_Population_Survey2018.json', 'Current_Population_Survey2019.json', 'Current_Population_Survey2020.json', 'Current_Population_Survey2021.json', 'Current_Population_Survey2022.json', 'Current_Population_Survey2023.json']


In [4]:
# realizamos la iteracion y conversion de los archivos en el lakehouse origen y los guardamos en el lakehouse destino
# tomando en cuenta si existen o no en el lakehouse destino, si no existen se realizara el proceso
# en caso contrario seguira la iteracion sin volver a reconvertir y guardar nuevamente el archivo
df_unido= None

for file in archivos_origen:
    if file not in archivos_destino:
        df = pd.read_json(f'/lakehouse/default/Files/CENSO/{file}')
        archivo = file.split('.')[0]
        df['Año_registro'] = archivo.split('y')[1]
        df.to_csv(f"abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/37a0382d-f783-4728-a697-48b1dc002937/Files/DATA FINAL/CENSO/{archivo}.csv", index=False)
        if df_unido is None:
            df_unido = df
            continue
        else:
            df_unido = pd.concat([df_unido, df])
            print(f'Se transformo y concateno {archivo}')
            continue
    else:
        print(f'Ya se realizo la conversion y guardado del archivo {file} en el lakehouse')
        continue
#Guardamos el archivo de todas las concatenaciones en el lakehouse destino
df_unido.to_csv(f"abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/Current_Population_Survey_unido.csv", index=False)
print('se guardo el archivo unido en lakehouse_destino')

StatementMeta(, , , Waiting, )

Se transformo y concateno Current_Population_Survey2012


Se transformo y concateno Current_Population_Survey2013


Se transformo y concateno Current_Population_Survey2014


Se transformo y concateno Current_Population_Survey2015


...

Se transformo y concateno Current_Population_Survey2018


Se transformo y concateno Current_Population_Survey2019
Se transformo y concateno Current_Population_Survey2020


Se transformo y concateno Current_Population_Survey2021


Se transformo y concateno Current_Population_Survey2022
Se transformo y concateno Current_Population_Survey2023


se guardo el archivo unido en lakehouse_destino


Realizamos un preprocesado de los valores de la columna GESTFIPS

In [2]:
df= pd.read_csv('abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/Current_Population_Survey_unido.csv', sep= ',')

StatementMeta(, cc516c66-db08-4daf-8e0e-e580de23a65f, 4, Finished, Available)

In [3]:
# instanciamos un diccionarios con los pares llave-valor que queremos reemplazar

dicc=  {21: "KY",
    51: "VA",
    15: "HI",
    26: "MI",
    45: "SC",
    55: "WI",
    36: "NY",
    44: "RI",
    31: "NE",
    22: "LA",
    42: "PA",
    47: "TN",
    25: "MA",
    27: "MN",
    30: "MT",
    41: "OR",
    53: "WA",
    1: "AL",
    33: "NH",
    23: "ME",
    56: "WY",
    16: "ID",
    18: "IN",
    9: "CT",
    10: "DE",
    6: "CA",
    35: "NM",
    40: "OK",
    46: "SD",
    50: "VT",
    13: "GA",
    20: "KS",
    49: "UT",
    8: "CO",
    32: "NV",
    37: "NC",
    28: "MS",
    34: "NJ",
    5: "AR",
    19: "IA",
    38: "ND",
    12: "FL",
    29: "MO",
    11: "DC",
    2: "AK",
    54: "WV",
    17: "IL",
    24: "MD",
    39: "OH",
    48: "TX",
    4: "AZ"}

StatementMeta(, cc516c66-db08-4daf-8e0e-e580de23a65f, 5, Finished, Available)

In [6]:
# modificacmos los valores de la columna reemplazandolos por sus respectivos valores en cada par llave valor del diccionario dicc

df['GESTFIPS'] = df['GESTFIPS'].replace(dicc)


StatementMeta(, cc516c66-db08-4daf-8e0e-e580de23a65f, 8, Finished, Available)

In [7]:
df.head()

StatementMeta(, cc516c66-db08-4daf-8e0e-e580de23a65f, 9, Finished, Available)

,HRNUMHOU,PTERNH1O,PEMNTVTY,PEFNTVTY,GTCBSASZ,GESTFIPS,GTCSA,PRCHLD,PRTAGE,PESEX,GTCBSA,HEFAMINC,HRHTYPE,state,Año_registro
0,2,-0.01,57,57,5,AL,0,4,40,1,13820,7,3,1,2011
1,2,-0.01,57,57,5,AL,0,0,16,2,13820,7,3,1,2011
2,4,-0.01,57,57,2,AL,0,0,53,2,11500,8,7,1,2011
3,4,-0.01,57,57,2,AL,0,0,52,1,11500,8,7,1,2011
4,4,13.60,57,57,2,AL,0,0,27,1,11500,8,7,1,2011


In [8]:
# Reemplazamos el archivo con los nuevos valores
df.to_csv('abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/Current_Population_Survey_unido.csv', index= False)

StatementMeta(, cc516c66-db08-4daf-8e0e-e580de23a65f, 10, Finished, Available)